In [194]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
import hvplot.pandas
from bokeh.sampledata.autompg import autompg_clean as df
import datetime as dt
import pytz

In [195]:
# Load the arestee.csv dataset.
df = pd.read_csv("IL/NIBRS_ARRESTEE.csv")
df.head()

,DATA_YEAR,ARRESTEE_ID,INCIDENT_ID,ARRESTEE_SEQ_NUM,ARREST_DATE,ARREST_TYPE_ID,MULTIPLE_INDICATOR,OFFENSE_TYPE_ID,AGE_ID,AGE_NUM,SEX_CODE,RACE_ID,ETHNICITY_ID,RESIDENT_CODE,UNDER_18_DISPOSITION_CODE,CLEARANCE_IND,AGE_RANGE_LOW_NUM,AGE_RANGE_HIGH_NUM
0,2019,33189213,111310423,1,01-MAR-19,2,N,16,5,13,F,2,2,R,R,NaN,13,0
1,2019,33182447,111310447,1,05-MAR-19,2,N,16,5,29,F,1,2,R,NaN,NaN,29,0
2,2019,33182461,111317969,1,08-MAR-19,1,N,16,5,14,M,2,2,R,R,NaN,14,0
3,2019,33189268,111310543,1,18-MAR-19,1,N,5,5,31,F,1,2,R,NaN,NaN,31,0
4,2019,33182483,111310559,1,19-MAR-19,3,N,16,5,44,M,2,2,N,NaN,NaN,44,0


In [196]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4140 entries, 0 to 4139
Data columns (total 18 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   DATA_YEAR                  4140 non-null   int64  
 1   ARRESTEE_ID                4140 non-null   int64  
 2   INCIDENT_ID                4140 non-null   int64  
 3   ARRESTEE_SEQ_NUM           4140 non-null   int64  
 4   ARREST_DATE                4140 non-null   object 
 5   ARREST_TYPE_ID             4140 non-null   int64  
 6   MULTIPLE_INDICATOR         4140 non-null   object 
 7   OFFENSE_TYPE_ID            4140 non-null   int64  
 8   AGE_ID                     4140 non-null   int64  
 9   AGE_NUM                    4140 non-null   int64  
 10  SEX_CODE                   4140 non-null   object 
 11  RACE_ID                    4140 non-null   int64  
 12  ETHNICITY_ID               4140 non-null   int64  
 13  RESIDENT_CODE              4140 non-null   objec

,DATA_YEAR,ARRESTEE_ID,INCIDENT_ID,ARRESTEE_SEQ_NUM,ARREST_TYPE_ID,OFFENSE_TYPE_ID,AGE_ID,AGE_NUM,RACE_ID,ETHNICITY_ID,CLEARANCE_IND,AGE_RANGE_LOW_NUM,AGE_RANGE_HIGH_NUM
count,4140.0,4.140000e+03,4.140000e+03,4140.000000,4140.000000,4140.000000,4140.0,4140.000000,4140.000000,4140.000000,0.0,4140.000000,4140.0
mean,2019.0,3.503123e+07,1.158294e+08,1.240580,1.677778,34.610628,5.0,31.238406,1.620531,1.935266,NaN,31.238406,0.0
std,0.0,1.494109e+06,3.848294e+06,1.290812,0.877927,16.292435,0.0,13.005841,0.558891,0.338641,NaN,13.005841,0.0
min,2019.0,3.197130e+07,1.073647e+08,1.000000,1.000000,2.000000,5.0,8.000000,0.000000,1.000000,NaN,8.000000,0.0
25%,2019.0,3.405283e+07,1.133848e+08,1.000000,1.000000,21.000000,5.0,21.000000,1.000000,2.000000,NaN,21.000000,0.0
50%,2019.0,3.482051e+07,1.157488e+08,1.000000,1.000000,39.000000,5.0,29.000000,2.000000,2.000000,NaN,29.000000,0.0
75%,2019.0,3.625525e+07,1.188924e+08,1.000000,3.000000,51.000000,5.0,39.000000,2.000000,2.000000,NaN,39.000000,0.0
max,2019.0,3.761454e+07,1.225162e+08,26.000000,3.000000,56.000000,5.0,83.000000,8.000000,3.000000,NaN,83.000000,0.0


In [197]:
df['ARREST_DATE'].map(lambda x: x.split('-')[1]).value_counts()

AUG    415
SEP    406
JUL    390
OCT    370
JAN    363
APR    354
JUN    337
MAY    330
NOV    324
DEC    303
MAR    302
FEB    246
Name: ARREST_DATE, dtype: int64

In [203]:
df['ARREST_DATE'] = pd.to_datetime(df['ARREST_DATE'])
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4140 entries, 0 to 4139
Data columns (total 18 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   DATA_YEAR                  4140 non-null   int64         
 1   ARRESTEE_ID                4140 non-null   int64         
 2   INCIDENT_ID                4140 non-null   int64         
 3   ARRESTEE_SEQ_NUM           4140 non-null   int64         
 4   ARREST_DATE                4140 non-null   datetime64[ns]
 5   ARREST_TYPE_ID             4140 non-null   int64         
 6   MULTIPLE_INDICATOR         4140 non-null   object        
 7   OFFENSE_TYPE_ID            4140 non-null   int64         
 8   AGE_ID                     4140 non-null   int64         
 9   AGE_NUM                    4140 non-null   int64         
 10  SEX_CODE                   4140 non-null   object        
 11  RACE_ID                    4140 non-null   int64         
 12  ETHNIC

,DATA_YEAR,ARRESTEE_ID,INCIDENT_ID,ARRESTEE_SEQ_NUM,ARREST_DATE,ARREST_TYPE_ID,MULTIPLE_INDICATOR,OFFENSE_TYPE_ID,AGE_ID,AGE_NUM,SEX_CODE,RACE_ID,ETHNICITY_ID,RESIDENT_CODE,UNDER_18_DISPOSITION_CODE,CLEARANCE_IND,AGE_RANGE_LOW_NUM,AGE_RANGE_HIGH_NUM
0,2019,33189213,111310423,1,2019-03-01,2,N,16,5,13,F,2,2,R,R,NaN,13,0
1,2019,33182447,111310447,1,2019-03-05,2,N,16,5,29,F,1,2,R,NaN,NaN,29,0
2,2019,33182461,111317969,1,2019-03-08,1,N,16,5,14,M,2,2,R,R,NaN,14,0
3,2019,33189268,111310543,1,2019-03-18,1,N,5,5,31,F,1,2,R,NaN,NaN,31,0
4,2019,33182483,111310559,1,2019-03-19,3,N,16,5,44,M,2,2,N,NaN,NaN,44,0


In [205]:
df['ARREST_DATE'] 

0      2019-03-01
1      2019-03-05
2      2019-03-08
3      2019-03-18
4      2019-03-19
          ...    
4135   2019-10-16
4136   2019-10-08
4137   2019-10-08
4138   2019-05-21
4139   2019-09-28
Name: ARREST_DATE, Length: 4140, dtype: datetime64[ns]

In [206]:
def get_season(date):
    if date > '2019-12-21':
        return "winter"
    elif date > '2019-06-21':
        return 'summer'
    elif date > '2019-03-21':
        return 'spring'
    else:
        return "fall"

In [211]:
# target but encode it df column arrest season
seasons = df["ARREST_DATE"].map(str).map(get_season).value_counts()
#df['ARREST_DATE'] = pd.to_datetime(df['ARREST_DATE'])
#df['ARREST_DATE']
seasons

summer    2230
spring    1020
fall       711
winter     179
Name: ARREST_DATE, dtype: int64

In [71]:
months = {"JAN" : "January", "FEB" : "February", "MAR" : "March", "APR" : "April", "MAY" : "May", "JUN" : "June", "JUL" : "July", "AUG" : "August", "SEP" : "September", "OCT" : "October", "NOV" : "November", "DEC" : "December"}
months["JAN"]

'January'

In [81]:
months_1 = {"JAN", "FEB", "MAR", "APR", "MAY", "JUN", "JUL", "AUG", "SEP", "OCT", "NOV", "DEC"}
for i in months_1:
    datetime.datetime.strptime(i,'%b').strftime('%B')
print(months_1)

{'JUN', 'JUL', 'FEB', 'AUG', 'NOV', 'MAR', 'DEC', 'SEP', 'MAY', 'JAN', 'APR', 'OCT'}


In [161]:
for i in df['ARREST_DATE']:
    datetime.datetime.strptime('i','%b').strftime('%B')

AttributeError: type object 'datetime.datetime' has no attribute 'datetime'

In [78]:
april = datetime.datetime.strptime('APR','%b').strftime('%B')
april

'April'

In [158]:
from datetime import date, datetime

Y = 2019 

seasons = [('winter', (date(Y,  1,  1),  date(Y,  3, 20))),
           ('spring', (date(Y,  3, 21),  date(Y,  6, 20))),
           ('summer', (date(Y,  6, 21),  date(Y,  9, 22))),
           ('autumn', (date(Y,  9, 23),  date(Y, 12, 20))),
           ('winter', (date(Y, 12, 21),  date(Y, 12, 31)))]

def get_season(now):
    if isinstance(now, datetime):
        now = now.date()
    now = now.replace(year=Y)
    return next(season for season, (start, end) in seasons
                if start <= now <= end)

# print(get_season(date.today()))
#for i in seasons:
#    print(i);
    

ValueError: Must have equal len keys and value when setting with an ndarray

In [19]:
import datetime
d = datetime.date(2019, 7 , 24)
print(d)

2019-07-24


In [20]:
today = datetime.date.today()
print(today)

2021-09-08


In [21]:
birthday = datetime.date(1993, 8, 1)
print(birthday)

1993-08-01


In [22]:
days_since_birth = (today - birthday).days
print(days_since_birth)

10265


In [23]:
tdelta = datetime.timedelta(days=10)
print(today - tdelta)

2021-08-29


In [25]:
print(today.month)

9


In [26]:
print(today.day)

8


In [28]:
print(today.weekday())

2


In [29]:
print(datetime.time(7, 2, 20, 15))

07:02:20.000015


In [30]:
# Add 10 hours to current day 
hour_delta = datetime.timedelta(hours=10)
print(datetime.datetime.now() + hour_delta)

2021-09-09 02:21:22.462935


In [37]:
datetime_today = datetime.datetime.now(tz=pytz.UTC)
datetime_pacific = datetime_today.astimezone(pytz.timezone('US/Pacific'))
print(datetime_pacific)
for tz in pytz.all_timezones:
    print(tz)

2021-09-08 13:29:26.343389-07:00
Africa/Abidjan
Africa/Accra
Africa/Addis_Ababa
Africa/Algiers
Africa/Asmara
Africa/Asmera
Africa/Bamako
Africa/Bangui
Africa/Banjul
Africa/Bissau
Africa/Blantyre
Africa/Brazzaville
Africa/Bujumbura
Africa/Cairo
Africa/Casablanca
Africa/Ceuta
Africa/Conakry
Africa/Dakar
Africa/Dar_es_Salaam
Africa/Djibouti
Africa/Douala
Africa/El_Aaiun
Africa/Freetown
Africa/Gaborone
Africa/Harare
Africa/Johannesburg
Africa/Juba
Africa/Kampala
Africa/Khartoum
Africa/Kigali
Africa/Kinshasa
Africa/Lagos
Africa/Libreville
Africa/Lome
Africa/Luanda
Africa/Lubumbashi
Africa/Lusaka
Africa/Malabo
Africa/Maputo
Africa/Maseru
Africa/Mbabane
Africa/Mogadishu
Africa/Monrovia
Africa/Nairobi
Africa/Ndjamena
Africa/Niamey
Africa/Nouakchott
Africa/Ouagadougou
Africa/Porto-Novo
Africa/Sao_Tome
Africa/Timbuktu
Africa/Tripoli
Africa/Tunis
Africa/Windhoek
America/Adak
America/Anchorage
America/Anguilla
America/Antigua
America/Araguaina
America/Argentina/Buenos_Aires
America/Argentina/Catam

In [43]:
# string formatting with dates
# 2019-03-09 -> March 3, 2019
print(datetime_pacific.strftime('%B %d %Y'))

September 08 2021
